In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18
np.random.seed(123)

In [2]:
df = pd.read_csv('./data/ch7_1_1_.csv', header=None)
df.columns=['data']

In [3]:
df

,data
0,1
1,0
2,0
3,3
4,0
5,0
6,0
7,0
8,0
9,1


In [14]:
stan_model = """
    data{
        int<lower=0> N;
        int<lower=0> X[N];
    }
    
    parameters{
        real<lower=0> lambda;
    }
        
    model{
        for(n in 1:N){
            X[n] ~ poisson(lambda);
        }
    }
    
    generated quantities{
        real<lower=0, upper=1> p;
        real sqrt_lambda;
#        p <- exp(-lambda) * pow(lambda, 2) * falling_factorial(1,2);
        p = (exp(-lambda ) * pow(lambda ,2) )/2; 
        sqrt_lambda = sqrt(lambda);
    }
"""

In [15]:
sm = pystan.StanModel(model_code = stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9f23412e32f37902b7924abc8cfbee52 NOW.


In [16]:
stan_data = {"N": df.shape[0], "X": df['data']}
fit = sm.sampling(data=stan_data, iter=11000, warmup=1000, chains=3, seed=1234)

In [17]:
fit

Inference for Stan model: anon_model_9f23412e32f37902b7924abc8cfbee52.
3 chains, each with iter=11000; warmup=1000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=30000.

              mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lambda         0.6  2.5e-3   0.24   0.22   0.42   0.57   0.74   1.16   9684    1.0
p              0.1  5.2e-4   0.05   0.02   0.06   0.09   0.13   0.21   9602    1.0
sqrt_lambda   0.76  1.6e-3   0.16   0.47   0.65   0.75   0.86   1.08   9222    1.0
lp__         -9.58  7.2e-3   0.73 -11.64  -9.74  -9.29  -9.12  -9.07  10294    1.0

Samples were drawn using NUTS at Tue Aug 14 13:53:55 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).